In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import urljoin

BASE = "https://emergencycaring.com/" 
OUTPUT = "er_waits.csv"

# Each table is inside a datawrapper with a unique ID. This function is extracting that ID by parsing the corresponding HTML code
def get_chart_id(state_url):
    r = requests.get(state_url)
    soup = BeautifulSoup(r.text, "html.parser")
    div = soup.find("div", id=lambda v: v and v.startswith("datawrapper-vis-"))
    if div:
        return div["id"].split("-")[-1]  # extract "m9pYd"
    return None

# Once we have the ID, we can directly fetch the dataset
def fetch_dataset(chart_id):
    csv_url = f"https://datawrapper.dwcdn.net/{chart_id}/1/dataset.csv"
    r = requests.get(csv_url)
    r.raise_for_status()
    return r.text

# Midwest states
states = ["illinois", "indiana", "iowa", "kansas", "michigan", "minnesota", "missouri", "nebraska", "north-dakota", "ohio", "south-dakota", "wisconsin"]

state_urls = [
    f"{BASE}/{state}-er-wait-times" for state in states 
]

# For each state, get the chart id, fetch the dataset, and create an entry in the final dictionary
state_dict = {}
for i in range(0,12):
    chart_id = get_chart_id(state_urls[i])
    if not chart_id:
        continue
    csv_text = fetch_dataset(chart_id)
    print("Got data for", state_urls[i], "chart:", chart_id)
    state_dict[states[i]] = csv_text


Got data for https://emergencycaring.com//illinois-er-wait-times chart: m9pYd
Got data for https://emergencycaring.com//indiana-er-wait-times chart: YgjCJ
Got data for https://emergencycaring.com//iowa-er-wait-times chart: Qz88s
Got data for https://emergencycaring.com//kansas-er-wait-times chart: d2Nxj
Got data for https://emergencycaring.com//michigan-er-wait-times chart: UMm3s
Got data for https://emergencycaring.com//minnesota-er-wait-times chart: XPxfy
Got data for https://emergencycaring.com//missouri-er-wait-times chart: q62JJ
Got data for https://emergencycaring.com//nebraska-er-wait-times chart: mikq8
Got data for https://emergencycaring.com//north-dakota-er-wait-times chart: jTZ9B
Got data for https://emergencycaring.com//ohio-er-wait-times chart: BHi5j
Got data for https://emergencycaring.com//south-dakota-er-wait-times chart: srO20
Got data for https://emergencycaring.com//wisconsin-er-wait-times chart: 4AeOf


In [13]:
import csv

# final CSV file
output_file = "emergencycaring_data.csv"

# open csv writer
with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)

    # header row 
    writer.writerow(["state", "Facility Name", "Municipality", "Expected Wait Time (Hours)", "201910 Wait Time (Hours)", "202407 Wait Time (Hours)"])

    # loop through states dictionary to create one csv with all ERs
    for state, table_text in state_dict.items():
        rows = table_text.strip().split("\n")  # split lines
        for row in rows[1:]:
            cols = row.split("\t")  # split columns
            writer.writerow([state] + cols)
